# **Import Libraries and import Dataset**

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Model

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

In [ ]:
import os
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
from google.colab import files
uploaded =files.upload()


In [ ]:
rawDataset =pd.read_csv('stroke.csv')
rawDataset.head()

In [ ]:
rawDataset.describe()

In [ ]:
# Drop the id column
rawDataset.drop(['id'], axis=1, inplace=True)

# **Checking Correlation**

In [ ]:
import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(10,10))
cor = rawDataset.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [ ]:
#Checking the distributions of all columns
figures = plt.figure(figsize = (20,25))
axis = figures.gca()
rawDataset.hist(ax = axis)

In [ ]:
sns.countplot(x='stroke',data=rawDataset)

In [ ]:
rawDataset.info()

# **Checking Null values**

In [ ]:
rawDataset.isnull().sum()

# **Replace Null Values With Mean Value**

In [ ]:
# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="train-stroke-prediction")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

In [ ]:
#replacing bmi column missing values with mean
mean_value_bmi=rawDataset['bmi'].mean()
rawDataset['bmi']=rawDataset['bmi'].fillna(mean_value_bmi)
rawDataset.isnull().sum()


In [ ]:
#replace yes and no with 1 and 0 in ever married column
rawDataset['ever_married'] = rawDataset['ever_married'].map({'Yes': 1, 'No': 0})
rawDataset['Residence_type'] = rawDataset['Residence_type'].map({'Urban': 1, 'Rural': 0})

In [ ]:
rawDataset = rawDataset.rename(columns={'Residence_type' : 'isUrban'})

#**Balancing the dataset using OverSampling**

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC

smotenc = SMOTENC(random_state=42, categorical_features=[0, 2, 3, 4, 5,6, 9])

In [ ]:
#Spliiting data into testing and training
Y = rawDataset['stroke']
X = rawDataset.drop(['stroke'], axis = 1)

In [ ]:
X, Y=smotenc.fit_resample(X,Y)

In [ ]:
Y_df = pd.DataFrame(Y, columns=['stroke'])

In [ ]:
X_df = pd.DataFrame(X, columns=rawDataset.columns.drop('stroke'))


In [ ]:
ds = pd.concat([X_df, Y_df], axis=1)

In [ ]:
sns.countplot(x='stroke',data=ds)

# **Onehot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
dataEncoded=enc.fit(ds[['gender','work_type','smoking_status']]).transform(ds[['gender','work_type','smoking_status']])

In [ ]:
temp=pd.DataFrame(dataEncoded.toarray(),columns=enc.get_feature_names())

In [ ]:
ds =pd.concat([temp,ds[['avg_glucose_level','isUrban','age','ever_married','hypertension','heart_disease','bmi', 'stroke']]], axis=1, join='inner')

In [ ]:
ds

# **Checking** **Outliers**

In [ ]:
sns.boxplot(x=rawDataset['bmi'])

In [ ]:
sns.boxplot(x=rawDataset['avg_glucose_level'])

In [ ]:
### Remove outliers based on the contamination value
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
iso = IsolationForest(max_samples='auto', contamination=float(0.1), n_estimators=100, random_state=rng)
yhat = iso.fit_predict(ds.iloc[:, 0:-1])
yhat.shape

In [ ]:
# select all rows that are not outliers
mask = yhat != -1
ds = ds.iloc[mask, :]
ds.shape

# **Spliting the training and testing sets**

In [ ]:
#Spliiting data into testing and training
Y = ds['stroke']
X = ds.drop(['stroke'], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=42)

# **Training dataset using  Different Models and Cheking Accuracy**

# SVM

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1, gamma=0.001, random_state=1)
model.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

y_predict = model.predict(X_test)
print("Accuracy:" ,accuracy_score(y_test, y_predict))
svm = accuracy_score(y_test, y_predict)

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn import metrics

Range_k = range(1,15)
scores = {}
scores_list = []
for k in Range_k:
   classifier = KNeighborsClassifier(n_neighbors=k)
   classifier.fit(X_train, y_train)
   y_pred3 = classifier.predict(X_test)
   scores[k] = metrics.accuracy_score(y_test,y_pred3)
   scores_list.append(metrics.accuracy_score(y_test,y_pred3))


print("Accuracy:",metrics.accuracy_score(y_test, y_pred3))
KNeighborsClassifier = metrics.accuracy_score(y_test, y_pred3)

# Gaussian Naive Bayes model

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred2 = gnb.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred2))
Gaussian_Naive_Bayes = metrics.accuracy_score(y_test, y_pred2)

# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50)
classifier.fit(X_train, y_train)

y_pred4 = classifier.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

 
print("Accuracy:",accuracy_score(y_test,y_pred4))
RandomForestClassifier = accuracy_score(y_test,y_pred4)

In [ ]:
modeles = ['SVM','KNeighborsClassifier','Gaussian_Naive_Bayes','RandomForestClassifier']
Accuracies = pd.DataFrame({"Model": modeles,"Accuracy": [svm,KNeighborsClassifier,Gaussian_Naive_Bayes,RandomForestClassifier]})
Accuracies

# **Hyper** **Parameter** **Tuning**

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')
#over=BorderlineSMOTE(sampling_strategy=0.3, k_neighbors=6)
#under=RandomUnderSampler(sampling_strategy=0.4)

from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold

# Number of trees in random forest
n_estimators = [200]
# Create the random grid
random_grid = {'n_estimators': n_estimators, 
               'min_samples_split': [1,2], 
               'min_samples_leaf': [1,2], 
               'max_features': ['sqrt','log2','auto'], 
               'max_depth': [10,20,30], 
               'bootstrap': [False,True]}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

cv=RepeatedStratifiedKFold(n_splits=7, n_repeats=2, random_state=42)

# Going to use RandomizedSearchCV because we have a lot of parameters and this way it will be more time efficient
grid_search=RandomizedSearchCV(estimator=rf
                               ,param_distributions=random_grid
                               ,n_jobs=-1
                               ,scoring='precision'
                               ,cv=cv
                               ,n_iter = 50
                              )

pipe=Pipeline([ ('model',grid_search)])

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
print("Training Accuracy :", pipe.score(X_train, y_train))

# **Model Accuracy Stats**

In [ ]:
# calculate accuracy
y_hat = pipe.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
#run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = pipe.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
#run.log('AUC', np.float(auc))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(
    cm,
    classes,
    normalize=False,
    title='Confusion matrix',
    cmap=plt.cm.Blues,
    ):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = ('.2f' if normalize else 'd')

    thresh = cm.max() / 2.

    for (i, j) in itertools.product(range(cm.shape[0]),
                                    range(cm.shape[1])):

        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment='center', color=('white' if cm[i,
                 j] > thresh else 'black'))

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_hat)
np.set_printoptions(precision=2)

# Plot confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['0', '1'],
                      title='Confusion matrix')

plt.rcParams['figure.figsize'] = (10, 10)


# **Model Deployment**

In [ ]:
# Save the trained model
model_file = 'stroke_model.pkl'
joblib.dump(value=pipe, filename=model_file)
run.upload_file(name = model_file, path_or_stream = './' + model_file)

In [ ]:
# Complete the run
run.complete()

# Register the model
run.register_model(model_path='stroke_model.pkl', model_name='stroke_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered sucessfully')

In [ ]:
model = ws.models['stroke_model']
print(model.name, 'version', model.version)

In [ ]:
import os

# Create a folder for the deployment files
deployment_folder = './stoke_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_stroke.py'
script_path = os.path.join(deployment_folder,script_file)

In [ ]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'stroke_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    data_values = {'x0_Female' : 0,
 'x0_Male':0,
 'x0_Other':0,
 'x1_Govt_job':0,
 'x1_Never_worked':0,
 'x1_Private':0,
 'x1_Self-employed':0,
 'x1_children':0,
 'x2_Unknown':0,
 'x2_formerly smoked':0,
 'x2_never smoked':0,
 'x2_smokes':0,
 'avg_glucose_level':0, 
 'isUrban':0, 
 'age':0, 
 'ever_married' : 0, 
 'hypertension' :0,
  'heart_disease' :0, 
  'bmi':0}
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']

    if data['gender'] == "Male":
        data_values['x0_Male'] = 1
    elif data['gender'] == "Female":
        data_values['x0_Female'] = 1
    else:   
        data_values['x0_Other'] = 1

    if data['work_type'] == "Government Job":
        data_values['x1_Govt_job'] = 1
    elif data['work_type'] == "Private Job":
        data_values['x1_Private'] = 1
    elif data['work_type'] == "Self Employed":
        data_values['x1_Self-employed'] = 1
    elif data['work_type'] == "Be with Children":
        data_values['x1_children'] = 1
    else:
        data_values['x1_Never_worked'] = 1

    if data['smoking_status'] == "Formerly Smoked":
        data_values['x2_formerly smoked'] = 1
    elif data['smoking_status'] == "Smokes":
        data_values['x2_smokes'] = 1
    else:
        data_values['x2_never smoked'] = 1

    data_values['age'] = data['age']
    data_values['avg_glucose_level'] = float(data['avg_glucose_level'])
    data_values['hypertension'] = 1 if data['hypertension'] == "Yes" else 0
    data_values['ever_married'] = 1 if data['ever_married'] == "Yes" else 0
    data_values['heart_disease'] = 1 if data['heart_disease'] == "Yes" else 0
    data_values['isUrban'] = 1 if data['isUrban'] == "Yes" else 0
    data_values['bmi'] = float(data['weight']) / (float(data['height'])**2)
    
    a = list(data_values.values())  

    # Get a prediction from the model
    predictions = model.predict([a])
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-stroke', 'stroke']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes[0])

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment(name='service-env')
python_packages = ['scikit-learn', 'azureml-defaults', 'azure-ml-api-sdk','imblearn', 'seaborn', 'matplotlib', 'pandas', 'scipy']
for package in python_packages:
    service_env.python.conda_dependencies.add_pip_package(package)
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "stroke-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)